In [1]:
import java.io.File
import scala.io.Source
import java.nio.file.Paths


import java.io.File
import scala.io.Source
import java.nio.file.Paths

In [2]:
import $ivy.`org.apache.spark::spark-sql:3.4.1`
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions.col
import org.apache.spark.sql.functions._

val spark = SparkSession.builder()
  .appName("Spark Notebook")
  .master("local[*]")
  .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

import spark.implicits._

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/07/25 08:21:24 WARN Utils: Your hostname, CALEB34NB resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/07/25 08:21:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/07/25 08:21:24 INFO SparkContext: Running Spark version 3.4.1
25/07/25 08:21:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/07/25 08:21:24 INFO ResourceUtils: ==============================================================
25/07/25 08:21:24 INFO ResourceUtils: No custom resources configured for spark.driver.
25/07/25 08:21:24 INFO ResourceUtils: ==============================================================
25/07/25 08:21:24 INFO SparkContext: Submitted application: Spark Notebook
25/07/25 08:21:24 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: co

import $ivy.$
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions.col
import org.apache.spark.sql.functions._
spark: SparkSession = org.apache.spark.sql.SparkSession@7f233fd7
import spark.implicits._

In [3]:
val v_path = Paths.get("").toAbsolutePath.normalize.toString().replace("notebooks","") + "input"
println(v_path)


/home/jrob/proj_de_scala_training/input


v_path: String = "/home/jrob/proj_de_scala_training/input"

In [4]:
val v_input_jobs = v_path + "/" + "jobs__1__1.csv"
println(v_input_jobs)

val df_jobs = (spark.read
               .option("header","false")
               .option("inferSchema","true")
               .csv(v_input_jobs).toDF("job_id","job_title")
)
df_jobs.show(3)

/home/jrob/proj_de_scala_training/input/jobs__1__1.csv
+------+-------------------+
|job_id|          job_title|
+------+-------------------+
|     1|Marketing Assistant|
|     2|           VP Sales|
|     3| Biostatistician IV|
+------+-------------------+
only showing top 3 rows



v_input_jobs: String = "/home/jrob/proj_de_scala_training/input/jobs__1__1.csv"
df_jobs: org.apache.spark.sql.package.DataFrame = [job_id: int, job_title: string]

In [5]:
val v_input_deps = v_path + "/" + "departments__1___1_.csv"
println(v_input_deps)

val df_deps = (spark.read
               .option("header","false")
               .option("inferSchema","true")
               .csv(v_input_deps).toDF("dept_id","dept_name")
)
df_deps.show(3)

/home/jrob/proj_de_scala_training/input/departments__1___1_.csv
+-------+--------------------+
|dept_id|           dept_name|
+-------+--------------------+
|      1|  Product Management|
|      2|               Sales|
|      3|Research and Deve...|
+-------+--------------------+
only showing top 3 rows



v_input_deps: String = "/home/jrob/proj_de_scala_training/input/departments__1___1_.csv"
df_deps: org.apache.spark.sql.package.DataFrame = [dept_id: int, dept_name: string]

In [6]:
val v_input_emps = v_path + "/" + "hired_employees__1___1_.csv"
println(v_input_jobs)

val df_emps = (spark.read
               .option("header","false")
               .option("inferSchema","true")
               .csv(v_input_emps).toDF("emp_id","emp_name","hire_date","dept_id","job_id")
)
df_emps.show(3)

/home/jrob/proj_de_scala_training/input/jobs__1__1.csv
+------+-----------+-------------------+-------+------+
|emp_id|   emp_name|          hire_date|dept_id|job_id|
+------+-----------+-------------------+-------+------+
|     1|Harold Vogt|2021-11-06 22:48:42|      2|    96|
|     2|   Ty Hofer|2021-05-30 01:43:46|      8|  null|
|     3|Lyman Hadye|2021-09-01 19:27:38|      5|    52|
+------+-----------+-------------------+-------+------+
only showing top 3 rows



v_input_emps: String = "/home/jrob/proj_de_scala_training/input/hired_employees__1___1_.csv"
df_emps: org.apache.spark.sql.package.DataFrame = [emp_id: int, emp_name: string ... 3 more fields]

In [7]:
import org.apache.spark.sql.functions.col

val df_factEmps = df_emps
  .join(df_jobs, Seq("job_id"), "left")
  .join(df_deps, Seq("dept_id"), "left")

df_factEmps.show(5)


+-------+------+------+--------------+-------------------+--------------------+---------------+
|dept_id|job_id|emp_id|      emp_name|          hire_date|           job_title|      dept_name|
+-------+------+------+--------------+-------------------+--------------------+---------------+
|      2|    96|     1|   Harold Vogt|2021-11-06 22:48:42|    Health Coach III|          Sales|
|      8|  null|     2|      Ty Hofer|2021-05-30 01:43:46|                null|        Support|
|      5|    52|     3|   Lyman Hadye|2021-09-01 19:27:38|Structural Analys...|    Engineering|
|     12|    71|     4| Lotti Crowthe|2021-10-01 09:04:21|     Statistician II|     Accounting|
|      6|    80|     5|Gretna Lording|2021-10-10 18:22:17|    Quality Engineer|Human Resources|
+-------+------+------+--------------+-------------------+--------------------+---------------+
only showing top 5 rows



import org.apache.spark.sql.functions.col
df_factEmps: org.apache.spark.sql.package.DataFrame = [dept_id: int, job_id: int ... 5 more fields]

In [8]:
// add a 0 value for the null cases
val df_factEmps_cleaned = df_factEmps.na.fill(Map(
  "job_id" -> 0,
  "dept_id" -> 0,
  "hire_date" -> "1900-01-01",  
  "job_title" -> "Not Found",
  "dept_name" -> "Not Found",
))

df_factEmps_cleaned.show(5)


+-------+------+------+--------------+-------------------+--------------------+---------------+
|dept_id|job_id|emp_id|      emp_name|          hire_date|           job_title|      dept_name|
+-------+------+------+--------------+-------------------+--------------------+---------------+
|      2|    96|     1|   Harold Vogt|2021-11-06 22:48:42|    Health Coach III|          Sales|
|      8|     0|     2|      Ty Hofer|2021-05-30 01:43:46|           Not Found|        Support|
|      5|    52|     3|   Lyman Hadye|2021-09-01 19:27:38|Structural Analys...|    Engineering|
|     12|    71|     4| Lotti Crowthe|2021-10-01 09:04:21|     Statistician II|     Accounting|
|      6|    80|     5|Gretna Lording|2021-10-10 18:22:17|    Quality Engineer|Human Resources|
+-------+------+------+--------------+-------------------+--------------------+---------------+
only showing top 5 rows



df_factEmps_cleaned: org.apache.spark.sql.package.DataFrame = [dept_id: int, job_id: int ... 5 more fields]

In [9]:
import org.apache.spark.sql.functions._

val df_factEmps_wrangled = df_factEmps_cleaned
  .withColumn("hire_dd", to_date(col("hire_date"), "yyyy-MM-dd"))
  .withColumn("hire_qq", quarter(col("hire_dd")))
  .withColumn("hire_yy", year(col("hire_dd")))
  .withColumn("hire_mm", month(col("hire_dd")))
  .withColumn("hire_period", year(col("hire_dd"))*100 + month(col("hire_dd")))

df_factEmps_wrangled.show(5)


+-------+------+------+--------------+-------------------+--------------------+---------------+----------+-------+-------+-------+-----------+
|dept_id|job_id|emp_id|      emp_name|          hire_date|           job_title|      dept_name|   hire_dd|hire_qq|hire_yy|hire_mm|hire_period|
+-------+------+------+--------------+-------------------+--------------------+---------------+----------+-------+-------+-------+-----------+
|      2|    96|     1|   Harold Vogt|2021-11-06 22:48:42|    Health Coach III|          Sales|2021-11-06|      4|   2021|     11|     202111|
|      8|     0|     2|      Ty Hofer|2021-05-30 01:43:46|           Not Found|        Support|2021-05-30|      2|   2021|      5|     202105|
|      5|    52|     3|   Lyman Hadye|2021-09-01 19:27:38|Structural Analys...|    Engineering|2021-09-01|      3|   2021|      9|     202109|
|     12|    71|     4| Lotti Crowthe|2021-10-01 09:04:21|     Statistician II|     Accounting|2021-10-01|      4|   2021|     10|     202110|

import org.apache.spark.sql.functions._
df_factEmps_wrangled: org.apache.spark.sql.package.DataFrame = [dept_id: int, job_id: int ... 10 more fields]

In [10]:
import org.apache.spark.sql.functions._

val df_factEmps_Qtr_Depts = df_factEmps_wrangled
  .groupBy(col("dept_name"), col("hire_yy"))
  .pivot("hire_qq", Seq(1, 2, 3, 4))
  .agg(count("emp_id"))
  .withColumn("total_hired", coalesce(col("1"), lit(0)) + coalesce(col("2"), lit(0)) + coalesce(col("3"), lit(0)) + coalesce(col("4"), lit(0)))
  .orderBy(col("total_hired").desc, col("dept_name"), col("hire_yy"))

df_factEmps_Qtr_Depts.show(5)


+--------------------+-------+---+---+---+---+-----------+
|           dept_name|hire_yy|  1|  2|  3|  4|total_hired|
+--------------------+-------+---+---+---+---+-----------+
|             Support|   2021| 34| 66| 59| 62|        221|
|         Engineering|   2021| 27| 47| 61| 73|        208|
|     Human Resources|   2021| 29| 56| 48| 72|        205|
|            Services|   2021| 30| 67| 57| 51|        205|
|Business Development|   2021| 27| 58| 49| 54|        188|
+--------------------+-------+---+---+---+---+-----------+
only showing top 5 rows



import org.apache.spark.sql.functions._
df_factEmps_Qtr_Depts: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [dept_name: string, hire_yy: int ... 5 more fields]

In [11]:
import org.apache.spark.sql.functions._

// Step 1: Filter for 2021
val df_2021 = df_factEmps_wrangled.filter(col("hire_yy") === 2021)

// Step 2: Count number of employees hired per department
val df_dept_hires = df_2021
  .groupBy(col("dept_id"), col("dept_name"))
  .agg(count("emp_id").alias("total_hired"))

// Step 3: Calculate the average hires across all departments
val mean_hires = df_dept_hires
  .agg(avg("total_hired"))
  .first()
  .getDouble(0) // this extracts the mean as a Double

// Step 4: Filter departments above the mean
val df_above_avg = df_dept_hires
  .filter(col("total_hired") > mean_hires)
  .orderBy(col("total_hired").desc)

df_above_avg.show(3,truncate = false)


+-------+-----------+-----------+
|dept_id|dept_name  |total_hired|
+-------+-----------+-----------+
|8      |Support    |221        |
|5      |Engineering|208        |
|7      |Services   |205        |
+-------+-----------+-----------+
only showing top 3 rows



import org.apache.spark.sql.functions._
df_2021: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [dept_id: int, job_id: int ... 10 more fields]
df_dept_hires: org.apache.spark.sql.package.DataFrame = [dept_id: int, dept_name: string ... 1 more field]
mean_hires: Double = 129.92307692307693
df_above_avg: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [dept_id: int, dept_name: string ... 1 more field]